# From OMOP to IDEA4RC Systemic treatment

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### From OMOP to IDEA4RC Systemic Treatment

In [ ]:
selectProcedures="""
SELECT 
    procedure_occurrence_id, procedure_concept_id, procedure_date, provider_id, procedure_end_date 
    FROM procedure_occurrence
    WHERE procedure_concept_id IN (4273629, 40310107, 912065);
"""

selectEpisodeData="""
    SELECT episode_concept_id, parent_episode_id,episode_id FROM episode WHERE episode_id IN (
    SELECT episode_id FROM episode_event WHERE event_id=%s AND episode_event_field_concept_id=1147082);
"""

selectDataParentEpisode="""
    SELECT episode_start_date FROM episode WHERE episode_id=%s;
"""

selectEpisodeEventIDEA="""
    SELECT id FROM episode_event WHERE cancer_episode IN (
    SELECT id FROM cancer_episode WHERE patient_id=%s AND date_of_diagnosis=%s);
"""

measurementsConceptIds={
    "number_of_cycles_administrations": None
}

observationConceptIds={
    "intent": (4179711, 4162591),
    "reason_for_end_of_treatment": (4082735, 438028, 4162706, 4105297, 37017062, 4306655)
}
procedures=curOMOP.fetchall()

insertTreatment="""
    INSERT INTO systemic_treatment (episode_event, hospital,type_of_systemic_treatment, start_date_systemic_treatment, end_date_systemic_treatment,number_of_cycles_administrations, treatment_response)
    VALUES (%s, %s, %s, %s, %s, %s, %s) RETURNING id;
"""
getCareSiteName="""
    SELECT care_site_name FROM care_site WHERE care_site_id=%s;
"""

ideaValues={
    4082735:'TREAT_COMPLETED',
    438028:'POIS_DRUG_MDE_SUBSTANCE',
    4162706:'CANC_TREATM_MORBID',
    4105297:'INTOLERANCE',
    37017062:'PROC_DISCONTINUED',
    4306655:'DEATH',
    32946:'COMPLETE_REMISSION',
    32947:'PARTIAL_REMISSION',
    32948:'STABLE_DISEASE',
    32949:'PROGRESSION',
    4273629:'CHEMOTHERAPY',
    40310107:'IMMUNOTHERAPY',
    912065:'TARGET_THERAPY',
    4119031:'PREOPERATIVE',
    4118656:'POST_OPERATIVE',
    3184898:'HYPERTHE_INTRAPERIT_CHEMO',
    4133895:'THERAPEUTIC',
    4179711:'PALLIATIVE',
    4162591:'CURATIVE',
    44804498:'ADJUVANT',
    44808409:'NEOADJUVANT'
}

getNumberCycles="""
    SELECT COUNT(*) FROM episode WHERE parent_episode_id=%s;
"""
#intent, setting, chemotherapy_info, reason_for_end_of_treatment,start_date_regimen_changed,end_date_regimen_changed
for procedure in procedures:
    curOMOP.execute(selectEpisodeData, (procedure[0]))
    episodeData=curOMOP.fetchone()
    curOMOP.execute(selectDataParentEpisode, (episodeData[1]))
    episodeStartDate=curOMOP.fetchone()
    curIDEA.execute(selectEpisodeEventIDEA, (episodeData[1], episodeStartDate[0]))
    episodeEventId=curIDEA.fetchone()
    curOMOP.execute(getCareSiteName, (procedure[3]))
    careSiteName=curOMOP.fetchone()
    curOMOP.execute(getNumberCycles, (episodeData[2]))
    numberCycles=curOMOP.fetchone()
    curIDEA.execute(insertTreatment, (episodeEventId[0], careSiteName[0], ideaValues[procedure[1]], procedure[2], procedure[4], numberCycles, episodeData[0]))
    treatmentId=curIDEA.fetchone()
    conn2.commit()
    for column in measurementsConceptIds.keys():
        values=measurementsConceptIds[column]
        queryGetMeasurement=f"""
            SELECT measuement_concept_id, value_as_number, value_as_concept_id FROM measurement WHERE measurement_concept_id IN {values} AND measurement_event_id={procedure[0]} AND meas_event_field_concept_id=1147082;
        """
        curOMOP.execute(queryGetMeasurement)
        measurement=curOMOP.fetchone()
        insert=f"""
            UPDATE radiotherapy SET {column}=%s WHERE id=%s;
        """
        valToInsert=None
        valToInsert=measurement[1]
        curIDEA.execute(insert, (valToInsert, treatmentId[0]))
        conn2.commit()
    for column in observationConceptIds.keys():
        selectObservation=f"""
            SELECT observation_concept_id, value_as_number, value_as_concept_id FROM observation WHERE observation_concept_id IN {observationConceptIds[column]} AND observation_event_id={procedure[0]} AND obs_event_field_concept_id=1147082;
        """
        curOMOP.execute(selectObservation)
        observation=curOMOP.fetchone()
        insert=f"""
            UPDATE radiotherapy SET {column}=%s WHERE id=%s;
        """
        valToInsert=ideaValues[observation[0]]
        curIDEA.execute(insert, (valToInsert, treatmentId[0]))
        conn2.commit()
    #Needs to be finished because still, does it actually make sense having 2 procedure occurrences? again the ata of the new systemic treatment is okay?